<a href="https://colab.research.google.com/github/Harshanand7/NLP/blob/main/GPT_NEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="niting3c/llama-2-7b-hf-zero-shot-prompt")

In [ ]:
from datasets import load_dataset

dataset = load_dataset("christophsonntag/OLID")

In [ ]:
df = dataset["train"].to_pandas()
df=df[['cleaned_tweet','subtask_a']]
df.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")


def classify_tweet(tweet):
    prompt = f"""
    A message can be classified as one of the following categories: Adversely Offensive, Mildly Offensive, Non Offensive.

    Based on these categories, classify this message:
    {tweet}
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_new_tokens=50)
    classification = tokenizer.decode(outputs[0], skip_special_tokens=True)

    classification_result = classification.split(prompt)[-1].strip()
    return classification_result


df['classification'] = df['cleaned_tweet'].apply(classify_tweet)


df.to_csv('classified_tweets.csv', index=False)